Version 1.0, 1-10-2025

# **2. Skript: Entfernen von Duplikaten**

Skript imports the xlsx from script 1, and consolidates as follows:

- removes all exact duplicates (same "Bemerkung" or same "Bemerkung" and "Textvorschlag" from same "Organisation", these are Stellungnahmen from the same organisation that were imported twice)
- consolidates duplicates from different organisations into one row (same "Bemerkung" or same "Bemerkung" and "Textvorschlag" from different "Organisation", the name of all organisations having the same comments is kept and copied into one row)

**Skript läuft in Azure Machine Learning Studio Empfohlene Compute-Umgebung: 16 Kerne, 64 GB RAM, 400 GB Festplatte (CPU) Kernel: Python 3.10 SDK v2**

**1. Zelle: Installationen von zusätzlichen Libraries**

In [ ]:
%pip install pandas==2.1.1 


**2. Zelle: Imports, define parameters, set hardcoded information**

In [ ]:
import pandas as pd

input_file = "/YOUR/INPUT/FOLDER/PATH/HERE/FILE.XLSX"
output_file = f'{input_file}_duplicates_removed.xlsx'
log_path = "duplicate_removal_log.txt"

**3. Zelle: Hauptskript**

In [ ]:
# works
# removes all exact copies in organization, artikel, absatz, buchstabe, Bemerkung and Textvorschlag =exakte duplikate, doppelte importe
# removes all exact copies within bemerkung and textvorschlag (= exakte duplikate, aber von verschiedenen Organisationen), schreibt kopierende Organisationen ein
# writes a log file

# Read all sheets from the Excel file into a dictionary of DataFrames
all_sheets = pd.read_excel(input_file, sheet_name=None)

output_sheets = {}
log_lines = []

for sheet_name, df in all_sheets.items():

    #First remove all exact duplicates (multiple imports)

    log_lines.append(f"Sheet: {sheet_name}")

    # Base subset columns for exact duplicate removal
    subset_cols = ["Organisation", "Bemerkung"]

    if "Textvorschlag" in df.columns:
        # When Textvorschlag exists, require these additional columns for duplicates to match
        subset_cols.extend(["Textvorschlag", "Artikel", "Absatz", "Buchstabe"])
    else:
        # If Textvorschlag does not exist, do not include those extra columns
        if "Textvorschlag" in subset_cols:
            subset_cols.remove("Textvorschlag")

    duplicates_mask = df.duplicated(subset=subset_cols, keep='first')
    exact_duplicates = df[duplicates_mask]
    num_exact_duplicates = len(exact_duplicates)
    log_lines.append(f"  Exact duplicate rows removed (by columns {subset_cols}): {num_exact_duplicates}")

    if num_exact_duplicates > 0:
        log_lines.append(f"  Exact duplicate row indices: {list(exact_duplicates.index)}")
        unique_orgs = exact_duplicates["Organisation"].unique()
        log_lines.append(f"  Organisations removed as duplicates: {list(unique_orgs)}")

    df_unique = df.drop_duplicates(subset=subset_cols, keep='first')

    # Check for required columns
    has_bemerkung = "Bemerkung" in df_unique.columns
    has_textvorschlag = "Textvorschlag" in df_unique.columns
    has_organisation = "Organisation" in df_unique.columns

    if not has_bemerkung or not has_organisation:
        log_lines.append("  Missing required columns for further processing. Skipping duplicate grouping.")
        output_sheets[sheet_name] = df_unique
        continue

    #Second, based on cleaned df from above, consolidate identical comments from different organizations    

    bemerkung_dict = {}
    # Count of groups by key, tracking number of total copies per group including original
    group_counts = {}

    for idx, row in df_unique.iterrows():
        key = (row["Bemerkung"], row["Textvorschlag"]) if has_textvorschlag else (row["Bemerkung"],)
        organisation = str(row["Organisation"])

        if key in bemerkung_dict:
            bemerkung_dict[key]["Organisation"] += "; " + organisation
            bemerkung_dict[key]["Count"] += 1
        else:
            bemerkung_dict[key] = {"Organisation": organisation, "Count": 1}

    # Track which keys appear more than once (duplicates removed)
    duplicate_keys = [k for k, v in bemerkung_dict.items() if v["Count"] > 1]
    log_lines.append(f"  Number of groups with duplicates removed: {len(duplicate_keys)}")
    if duplicate_keys:
        log_lines.append("  Duplicate group keys (Bemerkung, Textvorschlag if present):")
        for k in duplicate_keys:
            # Convert key to string and truncate to first 50 characters
            key_str = str(k)
            truncated_key = key_str[:50] + ("..." if len(key_str) > 50 else "")
            log_lines.append(f"    {truncated_key} - total copies: {bemerkung_dict[k]['Count']}")

    merged_data = []
    for idx, row in df_unique.iterrows():
        key = (row["Bemerkung"], row["Textvorschlag"]) if has_textvorschlag else (row["Bemerkung"],)
        if key in bemerkung_dict:
            new_row = row.copy()
            new_row["Organisation"] = bemerkung_dict[key]["Organisation"]
            new_row["Anzahl Kopien"] = bemerkung_dict[key]["Count"]
            merged_data.append(new_row)
            del bemerkung_dict[key]

    merged_df = pd.DataFrame(merged_data)

    # Insert "Anzahl Duplikate" after "Organisation"
    cols = list(merged_df.columns)
    if "Anzahl Kopien" in cols and "Organisation" in cols:
        org_idx = cols.index("Organisation")
        cols.insert(org_idx + 1, cols.pop(cols.index("Anzahl Kopien")))
        merged_df = merged_df[cols]

    output_sheets[sheet_name] = merged_df
    log_lines.append("")  # Empty line for sheet separation

# Write results to a new Excel file preserving sheet structure
with pd.ExcelWriter(output_file, engine="openpyxl") as writer:
    for sheet, data in output_sheets.items():
        data.to_excel(writer, sheet_name=sheet, index=False)

# Write log to file
with open(log_path, "w", encoding="utf-8") as f:
    f.write("\n".join(log_lines))

print(f"Processing done. Output saved to '{output_file}'.")
print(f"Duplicate removal log saved to '{log_path}'.")